# Benchmarks

## Initialize

In [ ]:
import os
import math
import pathlib
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import clear_output

import warnings
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [ ]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_medical_records"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

experiment = 220627
experiment_path = f"{output_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

In [ ]:
endpoint_defs = pd.read_feather(f"{output_path}/phecode_defs_220306.feather").sort_values("endpoint")

In [ ]:
data_outcomes = pd.read_feather(f"{output_path}/baseline_outcomes_220627.feather").set_index("eid")
endpoints = [e[:-6] for e in data_outcomes.columns if "_event" in e]

In [ ]:
data_outcomes

In [ ]:
eligable_eids = pd.read_feather(f"{output_path}/eligable_eids_220627.feather")
eids_dict = eligable_eids.set_index("endpoint")["eid_list"].to_dict()

In [ ]:
d = []
for endpoint in tqdm(endpoints):
    s = data_outcomes[f"{endpoint}_event"].loc[eids_dict[endpoint]]
    n = s.sum()
    freq = s.mean()
    d.append({"endpoint":endpoint, "eligable":len(eids_dict[endpoint]), "n": n, "freq": freq})

In [ ]:
endpoints_freqs = pd.DataFrame().from_dict(d).sort_values("endpoint").reset_index(drop=True)

In [ ]:
endpoints_freqs.sort_values("freq")#query("n>100")

In [ ]:
endpoints_ds = endpoints_freqs.query("n>100").sort_values("endpoint").reset_index(drop=True).merge(endpoint_defs)

In [ ]:
endpoints_ds 

In [ ]:
endpoints_ds.to_csv(f"{experiment_path}/endpoints.csv")

In [ ]:
endpoints_old = pd.read_csv(f"{output_path}/220531/endpoints.csv").endpoint.to_list()

In [ ]:
endpoints_ds.query("endpoint==@endpoints_old")